# Load various ML python modules and CMSSW into Jupyter kernel

NOTE: this kernel relies on the conda environment `myMLenvironment` built using the `build_conda_environment.ipynb`. If you haven't created that conda environment yet, do it before building this kernel. You do NOT need to re-build the conda environment every time (that's a very slow operation).



To teach Jupyter about a new environment, we will create a new `kernel`. In Jupyter, kernels are just execution environments - combinations of languages and libraries needed to run the code we want. 

So, for instance, one kernel could be a python kernel with CMSSW_9_1_0 and python2, while another would be CMSSW_12_0_2 with python3 and numpy. 

In this example we use CMSSW_12_0_2 and current versions of ML packages (numpy, tensorflow, keras) to build a new kernel called `MLpyROOT`.

To make the kernel, select the cell below and press `Shift+Enter`. Don't worry about the contents of the cell, this is simply code to make a CMSSW checkout, create a wrapper script, then teach Jupyter about that script.

In [4]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_12_0_2"
KERNEL_NAME="MLpyROOT"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ ! -d cmssw-env ]; then
    scramv1 project --name cmssw-env CMSSW $CMSSW_VER 
fi

# Now, install additional python packages
# First, setup the LMOD modules and activate the anaconda named environment:
source /etc/profile.d/modules.sh
module load anaconda/5.3.1-py37
source activate myMLenvironment

# Then setup the CMSSW environment
cd cmssw-env
eval `scramv1 runtime -sh`

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper2.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
pushd $DIR
eval `scramv1 runtime -sh`
popd
exec python3 "$@"
EOF
chmod +x bin/python_wrapper3.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper3.sh", 
  "-m", 
  "IPython.kernel", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"

Loaded CMSSW_12_0_2 into MLpyROOT_3!


If you see
```
Loaded CMSSW_12_0_2 into MLpyROOT
```
in the previous output, then you successfully loaded your kernel! Please select `File -> Close and Halt` from the menu to close this notebook, and proceed to the `exercises` notebook in the file browser